# Notebook

## 一些关于Dist

In [25]:
import torch
from torch.distributions import Normal

In [26]:
dist=Normal(loc=0,scale=2)

In [27]:
action=dist.sample()

In [28]:
entropy=dist.entropy()

In [31]:
dist.cdf(torch.tensor(1))

tensor(0.6915)

In [48]:
import torch
import torch.nn as nn
from torch.distributions import Categorical

# 定义 Actor 网络
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 64)
        self.fc2 = nn.Linear(64, action_dim)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = self.fc2(x)
        logits=torch.softmax(x,dim=-1)
        print(x,logits)
        dist = Categorical(logits=logits)
        return dist

# 创建 Actor 网络
actor = Actor(10, 4)

# 从 Actor 网络生成分布
state = torch.rand(1, 10)
dist = actor(state)

# 从分布中采样一个动作
action = dist.sample()

# 计算分布的熵
entropy = dist.entropy()


tensor([[-0.0560,  0.0762,  0.0540,  0.2109]], grad_fn=<AddmmBackward0>) tensor([[0.2191, 0.2501, 0.2446, 0.2862]], grad_fn=<SoftmaxBackward0>)


In [19]:
import torch
import torch.nn as nn
import network_sim
import gym

env=gym.make('PccNs-v0')
n_states=env.observation_space.shape[0]

# 定义模块列表
modules = [
    nn.Linear(n_states, 10,dtype=float),  # 输入大小为 1，输出大小为 10 的线性层
    nn.ReLU(),  # ReLU 激活函数
    nn.Linear(10, 1,dtype=float)  # 输入大小为 10，输出大小为 1 的线性层
]

# 创建顺序模型
model = nn.Sequential(*modules)

# 输入数据
state=env.reset()
input = torch.from_numpy(state) # 形状为 (批次大小, 输入大小) 的张量

# 前向传递
output = model(input)  # 形状为 (批次大小, 输出大小) 的张量


History length: 10
Features: ['sent latency inflation', 'latency ratio', 'send ratio']
Getting min obs for ['sent latency inflation', 'latency ratio', 'send ratio']
Reward: 0.00, Ewma Reward: 0.00


/home/data/envs/IL/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [18]:
output

tensor([-0.2016], dtype=torch.float64, grad_fn=<ViewBackward0>)

## 一些关于SB3

In [2]:
import gymnasium as gym

from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement

# Separate evaluation env
eval_env = gym.make("Pendulum-v1")
# Stop training if there is no improvement after more than 3 evaluations
stop_train_callback = StopTrainingOnNoModelImprovement(max_no_improvement_evals=3, min_evals=5, verbose=1)
eval_callback = EvalCallback(eval_env, eval_freq=1000, callback_after_eval=stop_train_callback, verbose=1)

model = SAC("MlpPolicy", "Pendulum-v1", learning_rate=1e-3, verbose=1)
# Almost infinite number of timesteps, but the training will stop early
# as soon as the the number of consecutive evaluations without model
# improvement is greater than 3
model.learn(int(1e10), callback=eval_callback,progress_bar=True)

Output()

Using cuda device
Creating environment from the given name 'Pendulum-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.68e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 245       |
|    time_elapsed    | 3         |
|    total_timesteps | 800       |
| train/             |           |
|    actor_loss      | 30.2      |
|    critic_loss     | 0.0579    |
|    ent_coef        | 0.502     |
|    ent_coef_loss   | -1.04     |
|    learning_rate   | 0.001     |
|    n_updates       | 699       |
----------------------------------


/home/data/envs/IL/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation 
environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and 
rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(

Eval num_timesteps=1000, episode_reward=-1714.01 +/- 48.42

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -465     |
| time/              |          |
|    episodes        | 52       |
|    fps             | 208      |
|    time_elapsed    | 49       |
|    total_timesteps | 10400    |
| train/             |          |
|    actor_loss      | 59.7     |
|    critic_loss     | 1.38     |
|    ent_coef        | 0.0622   |
|    ent_coef_loss   | -0.349   |
|    learning_rate   | 0.001    |
|    n_updates       | 10299    |
---------------------------------


Eval num_timesteps=11000, episode_reward=-158.59 +/- 149.09

Episode length: 200.00 +/- 0.00

---------------------------------
| eval/              |          |
|    mean_ep_length  | 200      |
|    mean_reward     | -159     |
| time/              |          |
|    total_timesteps | 11000    |
| train/             |          |
|    actor_loss      | 54.9     |
|    critic_loss     | 1.09     |
|    ent_coef        | 0.0897   |
|    ent_coef_loss   | -0.415   |
|    learning_rate   | 0.001    |
|    n_updates       | 10899    |
---------------------------------


Stopping training because there was no new best model in the last 4 evaluations